In [41]:
# change these to try this notebook out
# In "production", these will be replaced by the parameters passed to papermill
BUCKET = 'cse-linux-kubeflowpipelines-default'
PROJECT = 'cse-linux'
REGION = 'us'

In [42]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TRAIN'] = BUCKET+'/train/'
os.environ['DATA'] = BUCKET+'/rawdata'
os.environ['MODEL'] = BUCKET+'/model'

In [43]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


# Creating the input data pipeline

In [44]:
DATA_BUCKET = "gs://cse-linux-kubeflowpipelines-default/rawdata/"
#TRAIN_DATA_PATTERN = DATA_BUCKET + "train*"
#VALID_DATA_PATTERN = DATA_BUCKET + "test*"

In [45]:
!gsutil ls $DATA_BUCKET

gs://cse-linux-kubeflowpipelines-default/rawdata/
gs://cse-linux-kubeflowpipelines-default/rawdata/186_Pirates of the Caribbean.csv
gs://cse-linux-kubeflowpipelines-default/rawdata/229_Surrogates.csv
gs://cse-linux-kubeflowpipelines-default/rawdata/230_Swordfish.csv
gs://cse-linux-kubeflowpipelines-default/rawdata/231_Terminator.csv
gs://cse-linux-kubeflowpipelines-default/rawdata/51_conan_the_barbarian.csv


In [46]:
import os, json, math
import numpy as np
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.1


# train/train.py 테스트

In [47]:
os.environ['IMAGE_PREFIX'] = "cse-linux"
os.environ['TRAIN_DIR'] = "train"
os.environ['PROJECT_ID'] = "cse-linux"

In [95]:
!docker build -t $IMAGE_PREFIX-$TRAIN_DIR $TRAIN_DIR/.
!docker tag $IMAGE_PREFIX-$TRAIN_DIR:latest gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest
!docker push gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest

Sending build context to Docker daemon   34.3kB
Step 1/23 : FROM ubuntu:16.04
 ---> 9499db781771
Step 2/23 : RUN apt-get update
 ---> Using cache
 ---> e23217dd7c4b
Step 3/23 : RUN apt-get install -y software-properties-common vim
 ---> Using cache
 ---> 98af1b4d76ec
Step 4/23 : RUN add-apt-repository ppa:deadsnakes/ppa
 ---> Using cache
 ---> 98712b2f175e
Step 5/23 : RUN apt-get update
 ---> Using cache
 ---> 94c7ac45876e
Step 6/23 : RUN apt-get install -y build-essential python python-dev python3.6 python3.6-dev python3-pip python3.6-venv
 ---> Using cache
 ---> bf3097e869e5
Step 7/23 : RUN apt-get install -y git
 ---> Using cache
 ---> 20bb6257a069
Step 8/23 : RUN python3.6 -m pip install pip --upgrade
 ---> Using cache
 ---> 6d2a8a99dd78
Step 9/23 : RUN python3.6 -m pip install wheel
 ---> Using cache
 ---> d49ff1b94e43
Step 10/23 : RUN python3.6 -m pip install --upgrade google-cloud-storage
 ---> Using cache
 ---> bfbc53cfadb5
Step 11/23 : RUN python3.6 -m pip install --upgrade gc

In [96]:
!docker run gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest \
--csv_uri $BUCKET \
--checkpoint_uri gs://$MODEL/model.pkl \
--acc_uri gs://$BUCKET/latestacc/acc.csv \

File download Started... Wait for the job to complete.
Blobs: rawdata/186_Pirates of the Caribbean.csv
Blobs: rawdata/229_Surrogates.csv
Blobs: rawdata/230_Swordfish.csv
Blobs: rawdata/231_Terminator.csv
Blobs: rawdata/51_conan_the_barbarian.csv
9
Making model...
/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
No GPU available, using the CPU instead.
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes fo

In [97]:
!tar zcvf train.tar.gz *
!gsutil cp train.tar.gz gs://cse-linux-kubeflowpipelines-default/train

kfp_pipline.ipynb
pipeline.ipynb
setup.py
template.ipynb
train/
train/Dockerfile
train/.ipynb_checkpoints/
train/.ipynb_checkpoints/Dockerfile-checkpoint
train/.ipynb_checkpoints/train-checkpoint.py
train/.ipynb_checkpoints/train-Copy1-checkpoint.py
train/train-Copy1.py
train/train.py
train.tar.gz
train_pipelines.zip
Copying file://train.tar.gz [Content-Type=application/x-tar]...
/ [1 files][ 21.6 KiB/ 21.6 KiB]                                                
Operation completed over 1 objects/21.6 KiB.                                     
